In [1]:
import numpy as np
import matplotlib.pyplot as plt
import importlib
from EasyOptim import EasyPartition, EasyIteration, register_data
import neptune
from datetime import datetime

SyntaxError: invalid syntax (algo.py, line 66)

## Simulate phase conjugation optimization experiment
### Parameters of the system

In [ ]:
n_in = 200
n_out = 100

### Generate transmission matrix represented by a random Gaussian complex matrix

In [ ]:
H = np.random.randn(n_out,n_in)+1j*np.random.randn(n_out,n_in)

### Define the parameter to optimize: the energy at a target point

In [ ]:
def snr_on_target(i_target, phases):
    E_in = np.exp(1j*np.array(phases))
    E_out = H@E_in
    I_target = np.abs(E_out[i_target])**2
    #I_mean = np.mean(np.abs(E_out)**2)
    data = {'E_out':E_out}
    # return two objects:
    # - the value to optimize
    # - a dictionnary that will be saved, 
    #   the data corresponding to the best state will be returned by the algorithm
    return I_target, data

i_target = n_out//2
cost = lambda x: snr_on_target(i_target, x)

## Sequential algorithm

### Initialize the algorithm

In [ ]:
myalgo = EasyIteration(
    n_var = n_in,
    min_or_max = 'max',
    n_repeat = 2)


### Give the cost function to the algorithm object

In [ ]:
myalgo.register_callback(cost)

### Run the algorithm

It returns three objects:
* The best value found for the cost function,
* The corresponding input vector,
* A dictionnary of saved data (see the second example to populate this dicitonnary).

In [ ]:
phase_values = np.linspace(0,2*np.pi,10, endpoint = False)
best_value, phase_in_opt, data = myalgo.run(
    values = phase_values,
    x_init = None)

### Analalyze the results

**Compute output intensity for the optimize input field found**

In [ ]:
E_in_opt = np.exp(1j*np.array(phase_in_opt))
E_out_opt = H@E_in_opt
I_out_opt = np.abs(E_out_opt)**2
I_out_opt /= np.mean(I_out_opt)

**Compute output intensity for the ideal phase conjugation**

In [ ]:
# Ideal phase conjugation
E_in_cp = H[i_target,:].conjugate()
E_out_cp = H@E_in_cp
I_out_cp = np.abs(E_out_cp)**2
I_out_cp /= np.mean(I_out_cp)

In [ ]:
plt.figure(figsize = (12,5))
plt.subplot(121)
plt.plot(I_out_opt, label = 'Optimized')
plt.plot(I_out_cp, 'r--', label = 'Phase conjugation')
plt.title('Ouput intensity pattern')
plt.legend()
plt.subplot(122)
plt.plot(myalgo.evol)
plt.title('Evolution of the target value')

## Partition Algorithm

We will use here the callback functions that are called as specific times to log relevant data.

### Initialize a logger
Not compulsory, but it is a good practice and a good example of the use the callback functions.

We use Neptune.ai here, with the public account. Create your account to save privately your logs.

In [ ]:
neptune.init(
    api_token="ANONYMOUS",
    project_qualified_name = "shared/pytorch-lightning-integration"
)

In [ ]:
now = datetime.now().strftime("%d-%m-%Y %H:%M")
    
# these parameter will be saved by the logger when creating the experiment
params={'method':'partition',
        'start_time':now}

neptune.create_experiment(params = params)

**The link above gives access to the logs**

### Create a fraction function

It corresponds to the faction of element that will be updated at each iteration. 
The length of the vector give the number of iteration per repeat.

In [ ]:
frac_min = 1./n_in
fractions = np.linspace(.5, frac_min, n_in)
fractions = np.concatenate([fractions,np.array([frac_min]*n_in)])
plt.figure()
plt.plot(fractions)

### Create a custom optimization object

We overload the callback function to log data at different times.

Using the decorator `@EasyOptim.register_data`, the dictionnary returned by the functions are added to the one returned by the algorithm once finished.

In [ ]:
plt.ioff() 

class part_algo(EasyPartition):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
      
    @register_data
    def begin(self):
        return {'test_beginning':True,
                'initial_E_in': np.exp(1j*np.array(self.current_x))}
    
    @register_data
    def end(self):
        return {'test_end':42,
                'end_E_in': np.exp(1j*np.array(self.current_x))}
     
    @register_data
    def after_iter(self, i_iter, best_cost, best_x, best_data):
        neptune.log_metric('cost_evol',self.best_cost)
        if i_iter % 10 == 0:
            I_out = np.abs(best_data['E_out'])**2
            fig = plt.figure(figsize=(7, 9))
            plt.plot(I_out)
            neptune.log_image('I_out',fig)
            plt.close(fig)
        return {'test_iter':2}

### Initialize the algorithm object

In [ ]:
myalgo = part_algo(
        n_var = n_in,
        min_or_max = 'max',
        n_repeat = 1)
myalgo.register_callback(cost)

### Run the algorithm

In [ ]:
phase_values = np.linspace(0,2*np.pi,10, endpoint = False)
best_value, phase_in_opt, data = myalgo.run(
    values = phase_values,
    fractions = fractions,
    x_init = None)

### Look at the data dictionnary

In [ ]:
print(data.keys())

In [ ]:
plt.figure()
plt.title('Input phase')
plt.plot(np.angle(data['initial_E_in']), label = 'initial guess')
plt.plot(np.angle(data['end_E_in']), label = 'last (not best) iteration')
plt.legend()

### Analalyze the results

In [ ]:
E_in_cp = H[i_target,:].conjugate()
E_out_cp = H@E_in_cp
I_out_cp = np.abs(E_out_cp)**2
I_out_cp /= np.mean(I_out_cp)

E_in_opt = np.exp(1j*np.array(phase_in_opt))
E_out_opt = H@E_in_opt
I_out_opt = np.abs(E_out_opt)**2
I_out_opt /= np.mean(I_out_opt)

In [ ]:
plt.figure(figsize = (12,5))
plt.subplot(121)
plt.plot(I_out_opt, label = 'Optimized')
plt.plot(I_out_cp, 'r--', label = 'Phase conjugation')
plt.title('Ouput intensity pattern')
plt.legend()
plt.subplot(122)
plt.plot(myalgo.evol)
plt.title('Evolution of the target value')